# Ungga Challenge - Chatbot Asistente de agendación de citas

#### NOTA IMPORTANTE: De momento, el código que mejor muestra las capacidades del chatobt es el que se encuentra en la carpeta:

    ungga-bot-challenge/backend/codigo_chatbot.py

#### Esto es debido a que se hicieron múltiples pruebas del chatbot usando la consola. Y al final, para implementar el Frontend, hubo que modificar la estructura del chat, y debido al tiempo junto con los lentos periodos de ejecución, no se logró implementar en su totalidad (backend - frontend)


## 1. Instalar Ollama

#### Para poder ejecutar el chatbot, es requisito tener instalado Ollama de forma local en nuestro equipo. Este LLM se descarga desde la siguiente página:

####    https://ollama.com/download

#### Se descarga, se instala y luego en CMD se ejecutan el siguiente comando:

    Ollama run llama3

#### Esto descargará el modelo y lo ejecutará en un servidor local al cual se podrá acceder mediante API o de forma directa en LangChain

## 2. Pasos para ejecución del código (Backend)

### NOTA: Hay que asegurarse de tener una base de datos MongoDB conectada al servidor local, en donde se guardarán las citas

##### 1. Abrir carpeta '/ungga-bot-challenge/backend' en Visual Studio Code

##### 2. Abrir la terminal Git Bash y ejecutar el siguiente comando para activar el entorno virtual:
                source venv/Scripts/activate

##### 3. Con el entorno virtual activo, se descargan las librerías usando el siguiente comando (con ejecutarlo en el entorno virtual una sola vez ya es suficiente, y servirá tanto para main.py coom para codigo_chatbot.py.)
                pip install -r requirements.txt

##### 4. Una vez se hayan descargado las librerías, ejecutar el archivo main.py con el siguiente comando en la terminal de Git Bash:
                python main.py

##### 5. Con esto el código debería estar ejecutándose.

##### 6. (Extra) Para chatear con el bot desde la consola y ver todas sus capacidades, ejecutar el siguiente comando:
                python codigo_chatbot.py

## 3. Pasos para ejecución del código (Frontend)

##### 1. Abrir carpeta '/ungga-bot-challenge/frontend' en Visual Studio Code

##### 2. Abrir la terminal Git Bash y ejecutar el siguiente comando para activar los módulos de npm:
                npm install

##### 3. Con los módulos ya instalados, para iniciar el front en un servidor local se ejecuta el siguiente comando:
                npm run dev

##### 5. Con esto el entorno React debería estar ejecutándose.

# 2. Explicación del código (codigo_chatbot.py)

#### Primero debemos instalar las librerías necesarias

In [1]:
!pip install langchain langchain_community pymongo langgraph httpx -q
# httpx fue instalado por un error que se daba en la importación de módulos desde langgraph (hay que investigar ahí)

#### Luego importamos las funciones que usaremos:

In [2]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, MessageGraph

from pymongo import MongoClient                      # Para conectarnos con base de datos

#### Conexión con base de datos
##### Hay que asegurarse de tener conectada la base de datos a algún puerto


In [3]:
client = MongoClient('localhost', 27017)
database = client['Ungga-Challenge'] ### ------> Aqui debe insertar el nombre de la base de datos de MongoDBCompass local
collection = database['citas'] ### -----------> Aquí debe insertar el nombre de la colección dentro de la base de datos

#### Generamos un llm


In [4]:
llm = Ollama(model="llama3")

#### creamos un historial de chat vacío

In [5]:
chat_history = []

#### PromptTemplate para el Agente_de_Citas

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres un AI que agenda citas, me debes ir preguntando mi información hasta obtener lo siguiente:
            - nombre
            - email
            - horario de cita (día de la semana, día del mes, mes y hora), haz esto mensaje tras mensaje.
            
            Solamente cuando hayas obtenido todo, retornarás los datos en forma de diccionario: 
            'nombre': <nombre>, 'email': <email>, 'cita': <horario de la cita en MM/DD/2024 HH:MM>
            No retornes nada extra más que eso, solamente el diccionario, ninguna palabra más.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ])

#### Generamos la cadena

In [7]:
chain = prompt_template | llm

#### Función de Agente_de_citas
##### Nos agendará una cita en la base de datos


In [8]:
def chat(user_input):

    # Inicializamos "quiero agendar una cita" o algo parecido que incluya la palabra "cita"
    response = chain.invoke({"input": user_input, "chat_history": chat_history})
    print(response)
    chat_history.append(HumanMessage(content=response))  #siempre añadimos la info respectia al historial


    # Nos quedaremos en este loop hasta haber agendado la cita
    while True:
        pregunta = input("You: ")

        response = chain.invoke({"input": pregunta, "chat_history": chat_history})
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(SystemMessage(content=response))
        print("-"*50)
        print("AI: " + response)
        print("-"*50)

        datos_iniciales = response

        # Al recolectar todos los datos, devolverá un diccionario
        if datos_iniciales.startswith("{"):
            print("---------------out--------------")
            datos_diccionario = eval(datos_iniciales)
            print(type(datos_diccionario))
            print(datos_diccionario)
            user_nombre = datos_diccionario['nombre']
            print(user_nombre)
            user_email = datos_diccionario['email']
            print(user_email)
            user_cita = datos_diccionario['cita']
            print(user_cita)
            print("---------------out--------------")

            # Pregunta si queremos confirmar la cita
            confirmacion = "Ahora que tienes los datos, pregúntame si quiero confirmar la cita. Si te digo que no, pregúntame qué quiero modificar, lo modificas y vuelves a retornar el diccionario. Si te respondo afirmativamente, debes retornar la palabra True, nada más"
            response = chain.invoke({"input": confirmacion, "chat_history": chat_history}) 
            chat_history.append(HumanMessage(content=confirmacion)) 
            chat_history.append(SystemMessage(content=response))
            print("AI: " + response)   
            print("-"*50)

            pregunta = input("You: ") #aquí respondemos a si confirmamos o no
            response = chain.invoke({"input": pregunta, "chat_history": chat_history}) 
            chat_history.append(HumanMessage(content=pregunta)) 
            chat_history.append(SystemMessage(content=response))
            
            # Si confirmamos, se guardan los datos del diccionario en la base de datos
            if str(response) == "True":
                print("---------------out--------------")
                collection.insert_one({"name":user_nombre, "email": user_email, "cita": user_cita})
                print("cita agendada")
                print("---------------out--------------")
                return

#### Con Langgraph generamos un Grafo, que contiene al Agente_traductor en un nodo, la traducción en otro y un Edge que los conecta

In [9]:
graph = MessageGraph()

graph.add_node("oracle", llm)
graph.add_edge("oracle", END)

# Definimos punto de entrada
graph.set_entry_point("oracle")

# Compilamos el grafo junto con nuestro llm
runnable = graph.compile()

#### Generamos una función para cambiar entre agentes dependiendo de las palabras clave que digan:

In [10]:
def encontrar_palabras_clave(string, palabras):
    encontrar_palabras = []
    for palabra in palabras:
        if palabra in string:
            encontrar_palabras.append(palabra)
    return encontrar_palabras

#### Definimos el chat principal con el Agente_traductor

In [11]:
def chat_main(user_input):

    # definimos un historial de chat manual para el Agente Conversacional #3
    store1 = []
    user_input = ""
    
    # Este será la IA que reciba al usuario
    system_input = "Eres un chatbot que puede traducir lo que sea. También puedes agendar citas. Comienza dándome la bienvenida."
    store1.append("System: " + system_input)
    primer_mensaje = runnable.invoke(HumanMessage(system_input))
    store1.append("AI: " + primer_mensaje[-1].content)
    print("AI: " + primer_mensaje[-1].content)
    print("-"*50)
    
    # Aquí comienza el chat Loop, si el Usuario menciona "cita", se activará el agente de citas
    while user_input != "terminar":

        # user_input = input("Ingrese mensaje: ")
        user_input = input("You: ")
        store1.append("Human: " + user_input)
        palabras_clave_cita = ["cita", "citas", "appointment", "appointments"]
        iniciar_cita = encontrar_palabras_clave(user_input, palabras_clave_cita)

        # Si está escrita la palabra "cita", se inicia el chat con el otro agente
        if len(iniciar_cita)>0:
            chat(user_input)
            
        #Al finalizar el chat de agendar citas, se puede seguir conversando con el Agente Conversacional #3
        respuesta = runnable.invoke(HumanMessage(user_input))
        store1.append("AI: " + respuesta[-1].content)
        print("-"*50)
        print("AI: " + respuesta[-1].content)
        print("-"*50)


#### Finalmente iniciamos el código:

In [12]:
chat_main("hola")

AI: ¡Hola!

Me alegra poder saludarte y hacerme cargo de tu tiempo. Soy un chatbot altamente especializado en traducción, con capacidad para interpretar y responder a una variedad de preguntas y solicitudes.

Estoy aquí para ayudarte en todo lo que necesites, desde traducir textos o documentos hasta agendar citas y realizar tareas administrativas. No dudes en preguntarme algo, ¡estoy listo!

¿En qué puedo ayudarte hoy?
--------------------------------------------------


You:  hola, quiero saber cómo puedo decir "tengo sueño" en francés


--------------------------------------------------
AI: Hola!

La traducción de "tengo sueño" al francés es "j'ai sommeil".

* "Tengo" se traduce a "j'ai"
* "sueño" se traduce a "sommeil"

Por lo tanto, la respuesta correcta sería: "J'ai sommeil."

Si deseas decir "estoy cansado(a)" en francés, podrías usar la frase "J'ai besoin de sommeil" (necesito dormir) o simplemente "Je suis fatigué(e)" (estoy cansado(a)).
--------------------------------------------------


You:  how can i say "i'm hungry" in spanish?


--------------------------------------------------
AI: To say "I'm hungry" in Spanish, you can say:

Estoy hambriento(a).

Here's a breakdown of the sentence:

* Estoy means "I am"
* Hambriento(a) is the adjective that means "hungry"

Note: The verb "estar" (to be) is used to describe temporary or changing situations, such as hunger. If you're saying you're always hungry, you might use the verb "ser" (to be) instead:

Soy hambriento(a).

But if you're talking about a specific moment when you feel hungry, then "estar" is the way to go!

Hope that helps!
--------------------------------------------------


...

---------------out--------------
cita agendada
---------------out--------------
--------------------------------------------------
AI: ¡Excelente elección! Me alegra que estés interesado en programar una cita conmigo.

¿Cuál es el horario y fecha que te parece más conveniente para ti? Tenemos disponibles varios horarios y fechas, por lo que no hay problema en encontrar uno que se adapte a tus necesidades.

Además, ¿hay algún lugar o actividad específica que tengas en mente para nuestra cita? Me gustaría saber para poder planificar algo especial y personalizado para ti.
--------------------------------------------------


You:  ¿cuantos planetas hay en el sistema solar?


--------------------------------------------------
AI: Una pregunta clásica!

En el sistema solar, hay ocho planetas que se consideran "plenetes" (planets) reconocidos por la Unión Astronómica Internacional (UAI):

1. Mercurio
2. Venus
3. Tierra
4. Marte
5. Jupiter (Júpiter)
6. Saturno
7. Uranus
8. Neptuno

Además, existen otros cuerpos menores que orbitan alrededor del Sol, como:

* Los planetas enanos: Ceres (que es el más grande de los asteroides), Pluto y sus satélites, Eris, Haumea y Makemake.
* Los asteroides: Miles de pequeños objetos rocosos que orbitan entre Mercurio y Marte.
* Los cometas: Se cree que hay millones de cometas en el sistema solar, aunque solo unos pocos son visibles desde la Tierra.

En resumen, en el sistema solar hay ocho planetas "plenetes" reconocidos, pero muchos otros objetos menores que orbitan alrededor del Sol.
--------------------------------------------------


You:  terminar


--------------------------------------------------
AI: Entendi!

Você deseja que eu termine nossa conversa, não é?

Sinta-se à vontade para me dizer o motivo pelo qual você quer encerrar a nossa conversa. Se tiver alguma sugestão ou feedback para melhorar nossa interação, estou aqui para escutar!
--------------------------------------------------
